In [1]:
import numpy as np
import time as Time
from numba import jit
import matplotlib.pyplot as plt
import pyvista as pv
import torch

In [3]:
film = np.zeros((100, 100, 100, 2))

bottom = 80
film[:, :, 0:bottom, 0] = 10 # bottom

# height = 80

# film[:, :40, 0:height, 0] = 10
# film[:, 60:, 0:height, 0] = 10


In [4]:
sumFilm = np.sum(film, axis=-1)

print(sumFilm.shape)

(100, 100, 100)


In [5]:
# depo1 = torch.Tensor(np.logical_and(film[:, :, :] !=0, film[:, :, :] < 10)).to_sparse()
# depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(sumFilm[:, :, :]==10).to_sparse()
substrute = substrute.indices().numpy().T
# depomesh = pv.PolyData(depo1)
# depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
# depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

l:\program\Lib\site-packages\pyvista\core\utilities\points.py:52: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
l:\program\Lib\site-packages\pyvista\core\filters\data_set.py:2320: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:52649/index.html?ui=P_0x248f4848090_0&reconnect=auto" class="pyvis…

In [6]:
etchfilm = film

In [7]:
import Multi_Species_SF_etching

In [17]:
logname = 'Multi_species_benchmark_0729'
testEtch = Multi_Species_SF_etching.etching(mirror=True,inputMethod='bunch', pressure_pa=0.001, temperature=300, chamberSize=etchfilm.shape,
                                         depoThick=90, center_with_direction=np.array([[35,100,75]]), 
                                         range3D=np.array([[0, 70, 0, 100, 0, 150]]), InOrOut=[1], yield_hist=np.array([None]),
                                        reaction_type=False, param = [1.6, -0.7], N = 300000, 
                                        sub_xy=[0,0], film=etchfilm, n=1, cellSize=etchfilm.shape, 
                                        celllength=1e-5, kdtreeN=5, tstep=1e-5,
                                        substrateTop=40,posGeneratorType='benchmark', logname=logname)


# etching = testEtch.depo_position_increase_cosVel_normal(125, int(7e6), 2e-3, 5)


In [18]:
N = int(1e6)
velosity_matrix = np.zeros((N, 3))
tstep=1e-5
celllength=1e-5
# velosity_matrix[:, 0] = -1 * celllength /tstep
# velosity_matrix[:, 1] = -1 * celllength /tstep
velosity_matrix[:, 2] = -1 * celllength /tstep

typeID = np.zeros(N)

print(velosity_matrix[0])

[ 0.  0. -1.]


In [19]:
etching = testEtch.inputParticle(125, velosity_matrix, typeID, 2e-3, 10)

parcel (5000, 10)


running:   0%|                                                            | 0.00/100 [00:00<?, ?B/s]

parcel (5000, 10)
parcel (10000, 10)


running:   2%|█                                                    | 2.00/100 [00:00<00:21, 4.50B/s]

parcel (15000, 10)
parcel (20000, 10)
parcel (25000, 10)
parcel (30000, 10)


...

parcel (14662, 10)
after react
parcel (13873, 10)
after react


running:  31%|████████████████▍                                    | 31.0/100 [00:16<00:40, 1.68B/s]

parcel (13690, 10)
after react
parcel (13678, 10)
after react


running:  32%|████████████████▉                                    | 32.0/100 [00:16<00:36, 1.88B/s]

parcel (12944, 10)
depo finish


In [22]:
etching[0][50:60, 50:60, 82]

array([[[1.10123408, 0.        ],
        [1.04670308, 0.        ],
        [1.01760148, 0.        ],
        [1.02002594, 0.        ],
        [1.01187279, 0.        ],
        [1.0267537 , 0.        ],
        [1.01031516, 0.        ],
        [1.13451735, 0.        ],
        [1.19404016, 0.        ],
        [1.1464183 , 0.        ]],

       [[1.12379101, 0.        ],
        [1.12708704, 0.        ],
        [1.00761391, 0.        ],
        [1.17921094, 0.        ],
        [1.19435536, 0.        ],
        [1.1840703 , 0.        ],
        [1.19545944, 0.        ],
        [1.00607071, 0.        ],
        [1.01101505, 0.        ],
        [1.00477818, 0.        ]],

       [[1.01305059, 0.        ],
        [1.16636628, 0.        ],
        [1.17729348, 0.        ],
        [1.05383006, 0.        ],
        [1.18555137, 0.        ],
        [1.00127845, 0.        ],
        [1.18879077, 0.        ],
        [1.00146984, 0.        ],
        [1.00674611, 0.        ],
        [1

In [20]:
sumFilm = np.sum(etching[0], axis=-1)

depo1 = torch.Tensor(np.logical_and(sumFilm[:60, :, :,]!=10, sumFilm[:60, :, :,]!=0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(sumFilm[:60, :, :,]==10).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

l:\program\Lib\site-packages\pyvista\core\utilities\points.py:52: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
l:\program\Lib\site-packages\pyvista\core\filters\data_set.py:2320: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:52649/index.html?ui=P_0x2488d038810_3&reconnect=auto" class="pyvis…

In [12]:
sumFilm = np.sum(etching[0], axis=-1)

depo1 = torch.Tensor(np.logical_and(sumFilm[:, 50:, :,]!=10, sumFilm[:, 50:, :,]!=0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(sumFilm[:, 50:, :,]==10).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

l:\program\Lib\site-packages\pyvista\core\utilities\points.py:52: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
l:\program\Lib\site-packages\pyvista\core\filters\data_set.py:2320: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:50932/index.html?ui=P_0x19c0f17aa10_3&reconnect=auto" class="pyvis…

In [10]:
sumFilm = np.sum(testEtch.film, axis=-1)

depo1 = torch.Tensor(np.logical_and(sumFilm[:, :, :,]!=10, sumFilm[:, :, :,]!=0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(sumFilm[:, :, :,]==10).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

ValueError: Empty meshes cannot be plotted. Input mesh has zero points. To allow plotting empty meshes, set `pv.global_theme.allow_empty_mesh = True`

In [7]:
testEtch.parcel.shape

(7000000, 10)

In [14]:
testEtch.parcel[106]

array([161.24791799,  84.77586652, -82.53676749,   0.80623615,
         0.42387643,  -0.4126888 ,  68.        ,  58.        ,
        99.        ,   1.        ])

In [9]:
testEtch.parcel[100, :3] += testEtch.parcel[100, 3:6] 
print(testEtch.parcel[100])

[ 1.29022179e+02  1.60886601e+01 -1.53281836e+02  6.41898233e-01
  8.00382431e-02 -7.62601166e-01  6.30000000e+01  9.70000000e+01
  9.90000000e+01  1.00000000e+00]


In [10]:
testEtch.parcel[10000:11000][:,8].astype(int)


array([97, 98, 95, 98, 97, 98, 99, 99, 97, 99, 97, 99, 97, 95, 96, 97, 99,
       96, 97, 99, 95, 95, 99, 98, 95, 98, 96, 96, 97, 99, 97, 97, 97, 95,
       97, 97, 98, 96, 97, 98, 97, 96, 99, 97, 97, 95, 96, 99, 96, 98, 96,
       95, 99, 99, 96, 95, 97, 97, 96, 95, 95, 98, 95, 98, 95, 96, 98, 96,
       99, 98, 98, 97, 96, 99, 97, 96, 97, 96, 97, 98, 95, 99, 95, 99, 97,
       97, 98, 95, 99, 95, 97, 99, 97, 97, 96, 96, 95, 96, 95, 99, 96, 98,
       98, 95, 95, 99, 98, 97, 97, 96, 99, 98, 99, 97, 99, 95, 97, 98, 95,
       99, 95, 98, 96, 95, 98, 96, 96, 97, 97, 96, 97, 98, 97, 96, 99, 97,
       96, 97, 99, 99, 95, 96, 97, 98, 99, 95, 95, 97, 95, 98, 95, 99, 95,
       96, 99, 95, 98, 95, 97, 96, 95, 96, 97, 97, 99, 97, 98, 98, 99, 98,
       95, 97, 98, 98, 95, 97, 95, 97, 97, 99, 95, 99, 97, 98, 95, 96, 98,
       99, 96, 99, 98, 96, 99, 95, 95, 98, 98, 99, 99, 95, 95, 98, 99, 97,
       99, 99, 95, 98, 98, 98, 99, 97, 96, 97, 99, 96, 99, 95, 97, 96, 97,
       99, 96, 97, 96, 96